In [1]:
import json
import time

import requests
from bs4 import BeautifulSoup
import socks

import torch

class Article:
    def __init__(self, game_id: int, post_id: int, f_forum_id: int, subject: str, url: str, text: str, created_at: int, like_num: int, length: int):
        self.game_id = game_id
        self.post_id = post_id
        self.f_forum_id = f_forum_id
        self.subject = subject
        self.url = url
        self.text = text
        self.created_at = created_at
        self.like_num = like_num
        self.length = length

class Vector_Paragraph(Article):
    def __init__(self, game_id: int, post_id: int, f_forum_id: int, subject: str, url: str, text: str, created_at: int, like_num: int, length: int, vector: torch.Tensor):
        super().__init__(game_id, post_id, f_forum_id, subject, url, text, created_at, like_num, length)
        self.vector = vector
        self.text = text
        self.length = length

def Clawler(input_post_id, input_f_forum_id, ss_config = None):
    url = 'https://bbs-api.miyoushe.com/post/wapi/getPostFull'

    params = {
        'gids': '2',
        'post_id': input_post_id,
        'read': '1',
    }

    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Host': 'bbs-api.miyoushe.com',
        'Origin': 'https://www.miyoushe.com',
        'Referer': 'https://www.miyoushe.com/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Microsoft Edge";v="122"',
    }

    time.sleep(0.5)
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        html = response.text
    else:
        return None
    html = response.text
    data = json.loads(html)

    if data['data'] == None:
        return None

    content_html = data['data']['post']['post']['content']
    html_soup = BeautifulSoup(content_html, 'html.parser')
    text = html_soup.get_text(separator=" ", strip=True)
    game_id = data['data']['post']['post']['game_id']
    post_id = data['data']['post']['post']['post_id']
    f_forum_id = data['data']['post']['post']['f_forum_id']
    subject = data['data']['post']['post']['subject']
    url = 'https://www.miyoushe.com/ys/article/' + str(post_id)
    created_at = data['data']['post']['post']['created_at']
    like_num = data['data']['post']['stat']['like_num']
    length = len(text)

    target = Article(game_id, post_id, f_forum_id, subject,
                 url, text, created_at, like_num, length)

    if target.f_forum_id != input_f_forum_id:
        return None
    else:
        print('success!')
    return target

target = Clawler(50248063, 43)


success!


In [2]:
import re

import jieba
import tokenizer
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('all-MiniLM-L6-v2', local_files_only = True)
model = AutoModel.from_pretrained('all-MiniLM-L6-v2', local_files_only = True)

def split_text_to_chunks(text, max_length):
    chunks = []
    current_chunk = ""
    for char in text:
        current_chunk += char
        if len(current_chunk) >= max_length:
            last_natural_break = max(current_chunk.rfind(" "), current_chunk.rfind("。"), current_chunk.rfind("."), current_chunk.rfind("，"), current_chunk.rfind(","), current_chunk.rfind("（"), current_chunk.rfind("("))
            if last_natural_break > 0:
                chunks.append(current_chunk[:last_natural_break + 1])
                current_chunk = current_chunk[last_natural_break + 1:]
            else:
                chunks.append(current_chunk)
                current_chunk = ""
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

def remove_special_characters(text):
    pattern = re.compile(r'[^\w\u4e00-\u9fa5,.!?;:\'\"、，。？！；：“”‘’《》（）【】]')
    return re.sub(pattern, '', text)

def Tokenizer(text):
    text_cut = "".join(jieba.cut(text))
    
    inputs = tokenizer(text_cut, return_tensors="pt",
                    padding=True, truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(1)

    embeddings = embeddings.tolist()
    return embeddings

In [3]:
text = remove_special_characters(target.text)
chunks = split_text_to_chunks(text, 512)
cut_text = []
for i in chunks:
    data = Vector_Paragraph(target.game_id, target.post_id, target.f_forum_id, target.subject, target.url, i, target.created_at, target.like_num, len(i), Tokenizer(i))
    cut_text.append(data.__dict__)
data_index = []
data_index.append(cut_text)
print(data_index)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.604 seconds.
Prefix dict has been built successfully.


[[{'game_id': 2, 'post_id': '50248063', 'f_forum_id': 43, 'subject': '【V4.5攻略·角色攻略】千织·全面解析︱机制/养成/配队', 'url': 'https://www.miyoushe.com/ys/article/50248063', 'text': 'UID：100559319这里是阿斯盖特，本期是枫丹廷服装店“千织屋”的店主、来自稻妻的知名服装设计师千织的角色攻略！本期将详细地讲解千织的技能机制、装备与养成、相关队伍搭配内容为了优化阅读体验，已将部分文字内容置于折叠中，旅行者们可选择性查阅“我来决定自己该怎么走，要成为谁，没人能对我指手画脚。”【目录】01.简析一图流02.技能解析03.圣遗物搭配04.武器选择05.命之座解析06.队伍搭配【简析一图流】千织通过战技唤出人偶“袖”，可作为脱手后台输出手段。队友创造岩造物还将额外唤出一只人偶，增强战技输出能力。战技配合天赋的“再次点按战技键”可迅速切至下一名角色、提高队伍连招便捷性；点按普攻键，千织将获得岩元素附魔，使普攻、重击等可造成岩伤。通常作为后台岩伤输出，也可利用附魔机制体验前台输出玩法！文字太多不想看？先看看下面的一图流养成吧！【角色培养材料图表】以上是简析一图流部分技能升级：元素战技元素爆发普通攻击重击千织通常作为后台角色施放战技对敌人造成岩伤，优先升级元素战技；提高千织的爆发能力可升级元素爆发，最后升级普攻。另外，千织的6命会使普攻获得额外的防御倍率，升级普攻等级对伤害提升较小，', 'created_at': 1710294665, 'like_num': 2063, 'length': 505, 'vector': [[-0.07962040603160858, 0.05454440414905548, -0.06367256492376328, -0.11458712071180344, -0.16225427389144897, -0.07660577446222305, -0.07870076596736908, 0.013486037962138653, 0.017224106937646866, 0.04857330024242401, 0.29575949907302856, -0.1322042047977

IOStream.flush timed out


In [ ]:
from pymilvus import MilvusClient

CLUSTER_ENDPOINT="https://in01-6cef7f88da2c8c6.ali-cn-hangzhou.vectordb.zilliz.com.cn:19530" # Set your cluster endpoint
TOKEN="a8048e078be615e4f872f3e403f5b9f280a2332ab3fd4c81d57f06fb3cdbc2233b958a01bd4a12c35561fc0b5be124c2c380cd5c" # Set your token
COLLECTION_NAME="miyoushe_guide" # Set your collection name
# DATASET_PATH="data.json" # Set your dataset path

client = MilvusClient(
    uri=CLUSTER_ENDPOINT, # Cluster endpoint obtained from the console
    token=TOKEN # API key or a colon-separated cluster username and password
)

# Create a collection
client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=768
)

res = client.describe_collection(
    collection_name=COLLECTION_NAME,
    data = data_index
)
print(res)